In [1]:
import numpy as np
import pandas as pd

In [2]:
from collections import defaultdict

加载 line_profiler 扩增, 执行魔术命令lprun分析代码:
```jupyter
%load_ext line_profiler
%lprun
```
G20200616010153

In [3]:
%load_ext line_profiler

In [4]:
%load_ext Cython

## 概述
本练习尝试了4种方法对v1版本进行优化，并利用line_profiler, timeit工具进行测试对比。

| vx  |line_profiler              |timeit   | remark  |   |
|---|---|---|---|---|
|target_mean_v1  | 33.85s         |17.2s   | 优化前  |   |
|target_mean_v2   |2.04s / 17+ up |864ms / 19+ up   | v1基础上，分组循环改进，减少重复计算   |   |
|target_mean_v3   |0.039s / 867+up|7.76ms / 2216+ up   | v2基础上，去dataframe检索  |   |
|target_mean_v4   |0.031s / 1000+ |12.4ms / 1387+ up   | v3基础上，分组统计，字典优化 |   |
|target_mean_v5   |  -- | 877us / 21000+up   |v3基础上，cython改进   |   |

小结：
* pandas的检索实在太慢
* defaultdict的访问性能 已经接近数组的随机存取。
* 用C/C++中的静态类型来优化python中的动态类型，性能大幅度提速 target_mean_v5

遗留问题：
line_profiler, timeit两种不同工具的测试结果不相同？

In [5]:
# test data
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [6]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [7]:
%lprun -f target_mean_v1 target_mean_v1(data, 'y', 'x')

Timer unit: 1e-06 s

Total time: 38.3201 s
File: <ipython-input-6-1e10119a1d07>
Function: target_mean_v1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def target_mean_v1(data, y_name, x_name):
     2         1        243.0    243.0      0.0      result = np.zeros(data.shape[0])
     3      5001       6766.0      1.4      0.0      for i in range(data.shape[0]):
     4      5000   32304996.0   6461.0     84.3          groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
     5      5000    6008104.0   1201.6     15.7          result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
     6         1          0.0      0.0      0.0      return result

## target_mean_v2
target_mean_v1 中为每条记录求mean,count，会导致其余数据多次sum, count。

改进方法：
* 先对所有数据分组，同时计算该分组类的count，sum.
* 在依次处理每条记录：找到该记录对应组，减去记录本身值后求mean。

改进前时间复杂度： O(n*n), 改进后：O(n)，应该要快不少。

In [16]:
def target_mean_v2(data, y_name, x_name):
    grp_sum = defaultdict(lambda: 0)
    grp_cnt = defaultdict(lambda: 0)
    nrow = data.shape[0]
    result = np.zeros(nrow)
    total_sum = 0
    total_count = 0
    for rx in range(nrow):
        row = data.iloc[rx]
        x_v, y_v = row[x_name], row[y_name]
        grp_sum[x_v] += y_v
        grp_cnt[x_v] += 1
        
        total_sum += y_v
        total_count += 1
        
    total_mean = total_sum / total_count
    
    for rx in range(nrow):
        row = data.iloc[rx]
        x_v, y_v = row[x_name], row[y_name]
        g = grp_cnt[x_v]
        if g == 1:
            result[rx] = total_mean
        else:
            result[rx] = (grp_sum[x_v] - y_v)/(g - 1)
    return result

In [37]:
%lprun -f target_mean_v2 target_mean_v2(data, 'y', 'x')

Timer unit: 1e-06 s

Total time: 2.04039 s
File: <ipython-input-36-c2315dfff126>
Function: target_mean_v2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def target_mean_v2(data, y_name, x_name):
     2         1          4.0      4.0      0.0      grp_sum = defaultdict(lambda: 0)
     3         1          1.0      1.0      0.0      grp_cnt = defaultdict(lambda: 0)
     4         1         21.0     21.0      0.0      nrow = data.shape[0]
     5         1         23.0     23.0      0.0      result = np.zeros(nrow)
     6         1          1.0      1.0      0.0      total_sum = 0
     7         1          1.0      1.0      0.0      total_count = 0
     8      5001       3033.0      0.6      0.1      for rx in range(nrow):
     9      5000     895742.0    179.1     43.9          row = data.iloc[rx]
    10      5000     121521.0     24.3      6.0          x_v, y_v = row[x_name], row[y_name]
    11      5000       8

## target_encoding_v3
target_encoding_v2 比 v1快约17倍。 
根据v2的trick分析知： data.iloc 语句还有待优化: 即采用DataFrame原生的元素检索方式性能非得低下，两处循环中DataFrame检索消耗占到了98%。

利用DataFrame的values属性可以获得DataFrame内部数据的Numpy.narray二位数组表示形式。

In [17]:
def target_mean_v3(data, y_name, x_name):
    grp_sum = defaultdict(lambda: 0)
    grp_cnt = defaultdict(lambda: 0)
    nrow = data.shape[0]
    result = np.zeros(nrow)
    total_sum = 0
    total_count = 0
    
    x = data[x_name].values
    y = data[y_name].values
    
    xy = zip(x, y)
    
    for x_v, y_v in xy:
        grp_sum[x_v] += y_v
        grp_cnt[x_v] += 1
        
        total_sum += y_v
        total_count += 1
        
    total_mean = total_sum / total_count
    
    return np.array([total_mean if grp_cnt[x_v] <= 1 else (grp_sum[x_v] - y_v)/(grp_cnt[x_v] - 1) for x_v, y_v in zip(x, y) ])

In [60]:
%lprun -f target_mean_v3 target_mean_v3(data, 'y', 'x')

Timer unit: 1e-06 s

Total time: 0.039221 s
File: <ipython-input-59-6ce07509522c>
Function: target_mean_v3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def target_mean_v3(data, y_name, x_name):
     2         1          5.0      5.0      0.0      grp_sum = defaultdict(lambda: 0)
     3         1          3.0      3.0      0.0      grp_cnt = defaultdict(lambda: 0)
     4         1         19.0     19.0      0.0      nrow = data.shape[0]
     5         1         18.0     18.0      0.0      result = np.zeros(nrow)
     6         1          2.0      2.0      0.0      total_sum = 0
     7         1          2.0      2.0      0.0      total_count = 0
     8                                               
     9         1         60.0     60.0      0.2      x = data[x_name].values
    10         1         25.0     25.0      0.1      y = data[y_name].values
    11                                               
    12 

## target_encoding_v4

v3 相比 v1提升了 1100倍（33 / 0.03）。从trick结果来看，如果不采用C语言等其他技术，那么尝试用对字典的查找进行优化。例如，采用数组的来存放分组（用下标来标识组key）


In [18]:
def target_mean_v4(data, y_name, x_name):
    nrow = data.shape[0]
    grp_sum = np.zeros(nrow)
    grp_cnt = np.zeros(nrow)
    result = np.zeros(nrow)
    total_sum = 0
    total_count = 0
    
    x = data[x_name].values
    y = data[y_name].values
    
    xy = zip(x, y)
    
    for x_v, y_v in xy:
        grp_sum[x_v] += y_v
        grp_cnt[x_v] += 1
        
        total_sum += y_v
        total_count += 1
        
    total_mean = total_sum / total_count
    
    return np.array([total_mean if grp_cnt[x_v] <= 1 else (grp_sum[x_v] - y_v)/(grp_cnt[x_v] - 1) for x_v, y_v in zip(x, y) ])

In [65]:
%lprun -f target_mean_v4 target_mean_v4(data, 'y', 'x')

Timer unit: 1e-06 s

Total time: 0.031409 s
File: <ipython-input-64-10b9a7076fd7>
Function: target_mean_v4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def target_mean_v4(data, y_name, x_name):
     2         1         16.0     16.0      0.1      nrow = data.shape[0]
     3         1        100.0    100.0      0.3      grp_sum = np.zeros(nrow)
     4         1         71.0     71.0      0.2      grp_cnt = np.zeros(nrow)
     5         1         93.0     93.0      0.3      result = np.zeros(nrow)
     6         1          1.0      1.0      0.0      total_sum = 0
     7         1          1.0      1.0      0.0      total_count = 0
     8                                               
     9         1         50.0     50.0      0.2      x = data[x_name].values
    10         1         18.0     18.0      0.1      y = data[y_name].values
    11                                               
    12         1       

## target_encoding_v5
v4与v3相比，性能上没有显著提升。 说明 字典的查找并非瓶颈。 这么看来应该要优化内存的读写了。

In [48]:
%%cython
import numpy as np
cimport numpy as cnp
from collections import defaultdict
from libc.string cimport memset

def default_value():
    return 0

cpdef target_mean_v5(cnp.ndarray[long] x, cnp.ndarray[double] y, cnp.ndarray[double] result, const int nlabels):
    n = x.shape[0]
    #sum_dict = defaultdict(default_value)
    #cnt_dict = defaultdict(default_value)
    cdef double sum_dict[nlabels]
    cdef long cnt_dict[nlabels]
    cdef double total = 0
    cdef long cnt = 0
    
    memset(sum_dict, 0, nlabels*sizeof(double))
    memset(cnt_dict, 0, nlabels*sizeof(long))
    
    for i in range(n):
        xv, yv = x[i], y[i]
        sum_dict[xv] += yv
        cnt_dict[xv] +=1
        total += yv
        cnt +=1
        
    total_mean = total / cnt
    for i in range(n):
        xv, yv = x[i], y[i]
        c = cnt_dict[xv]
        result[i] = total_mean if c == 1 else (sum_dict[xv] - yv)/(c-1)
    return result

In [58]:
r1 = target_mean_v1(data, 'y', 'x')

In [13]:
%%timeit
r=target_mean_v1(data, 'y', 'x')

18.8 s ± 1.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
r2=target_mean_v2(data, 'y', 'x')

838 ms ± 35.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
r3=target_mean_v3(data, 'y', 'x')

7.16 ms ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%timeit
r4=target_mean_v4(data, 'y', 'x')

12.4 ms ± 76.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
%%timeit
r5 = target_mean_v5(data['x'].to_numpy(np.int), data['y'].to_numpy(np.float64), np.zeros(data.shape[0], dtype=np.float64), 10)

877 µs ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [50]:
# r2 = target_mean_v2(data, 'y', 'x')
r3 = target_mean_v3(data, 'y', 'x')
# r4 = target_mean_v4(data, 'y', 'x')
r5 = target_mean_v5(data['x'].to_numpy(np.int), data['y'].to_numpy(np.float64), np.zeros(data.shape[0], dtype=np.float64), 10)
np.linalg.norm(r5 - r3)

0.0